Below is TOPSIS method


In [105]:
import numpy as np
import pandas as pd
a = np.array([[1,2,3],[4,5,6]])
a.sum(axis = 0)
from to_latex_table import array_latex_table, csv_latex_table


ModuleNotFoundError: No module named 'to_latex_table'

**==========evaluation methods================**

In [ ]:
def topsis(matrix, weights, criteria_types):
    """
    Perform TOPSIS ranking.
    
    Parameters
    ----------
    matrix : 2D array-like
        Decision matrix with shape (m, n) — m alternatives, n criteria.
    weights : 1D array-like
        Importance weights for each criterion (should sum to 1).
    criteria_types : list of str
        Each element is 'benefit' or 'cost' indicating the type of criterion.
    
    Returns
    ----------
    scores : ndarray
        Closeness coefficient for each alternative (higher = better).
    rankings : ndarray
        Indices of alternatives ranked from best to worst.
    """
    # Avoid wrong imports
    
    if sum(weights) != 1:
        raise ValueError("Weights must sum to 1.")
    if len(weights) != matrix.shape[1]:
        raise ValueError("Number of weights must match number of criteria.")
    
    # Step 1: Normalize the decision matrix
    norm_matrix = matrix / np.sqrt((matrix ** 2).sum())
    print(f"norm_matrix = {norm_matrix}")
    
    # Step 2: Multiply by weights
    weighted_matrix = norm_matrix * weights

    # Step 3: Determine ideal and negative-ideal solutions
    ideal_solution = []
    negative_ideal_solution = []
    for i in range(weighted_matrix.shape[1]):
        if criteria_types[i] == '+':
            ideal_solution.append(weighted_matrix[:, i].max())
            negative_ideal_solution.append(weighted_matrix[:, i].min())
        else:
            ideal_solution.append(weighted_matrix[:, i].min())
            negative_ideal_solution.append(weighted_matrix[:, i].max())
    
    ideal_solution = np.array(ideal_solution)
    negative_ideal_solution = np.array(negative_ideal_solution)

    # Step 4: Calculate distances to ideal and negative-ideal solutions
    distance_to_ideal = np.sqrt(((weighted_matrix - ideal_solution) ** 2).sum(axis=1))
    distance_to_negative_ideal = np.sqrt(((weighted_matrix - negative_ideal_solution) ** 2).sum(axis=1))

    # Step 5: Calculate performance score
    performance_score = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)

    return performance_score

In [ ]:
# from asyncio import log

def entropy(infor):
    """

    Args:
        infor (np.array): input array[1,n]

    Returns:
        float: entropy value
    """
    if type(infor)!= np.array:
        infor = np.array(infor)
    return np.sum(-infor * np.log(infor + np.finfo(float).eps)) / np.log(len(infor))


# Data Standardization
def cost_indicator(matrix):
    """
    Args:
        matrix (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: cost indicator matrix
    """
    if type(matrix)!= np.array:
        matrix = np.array(matrix)
    ind = np.argmax(matrix, axis = 1)
    maxi = matrix[ind, :]
    print(maxi)
    print(np.sum(maxi - matrix, axis = 0))
    matrix = (maxi - matrix) / np.sum(maxi - matrix, axis = 0)
    return matrix

def benefit_indicator(matrix):
    """
    Args:
        matrix (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: benefit indicator matrix
    """
    if type(matrix)!= np.array:
        matrix = np.array(matrix)

    return matrix / np.sum(matrix, axis = 0)

def optimal_value_indicator(matrix, optimal_value):
    """
    Args:
        matrix (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: benefit indicator matrix
    """
    if type(matrix)!= np.array:
        matrix = np.array(matrix)
    if optimal_value is None:
        optimal_value = np.median(matrix, axis=0)
    maxn = np.max(abs(matrix - optimal_value))
    if maxn == 0:
        maxn = np.finfo(float).eps
    matrix = 1 - (abs(matrix - optimal_value) / maxn)
    matrix = matrix / np.sum(matrix, axis = 0)
    return matrix


# Calculate Weights
def entropy_weights(matrix, indicator, optimal_value = None):
    """
    Args:
        matrix (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: entropy weights (1,n)
    """
    if type(matrix)!= np.array:
        matrix = np.array(matrix)
    # Normalization
    #matrix = matrix / matrix.sum(axis=0)
    if indicator is optimal_value_indicator:
        matrix = indicator(matrix, optimal_value)
    else:
        matrix = indicator(matrix)
    #print(matrix)
    matrix = np.where(matrix == 0, np.finfo(float).eps, matrix)
    # Entropy calculation
    n_criteria = matrix.shape[1]
    
    entropy_values = np.array(list(map(entropy, matrix.T)))
   
    #print(entropy_values)
    # Degree of diversification
    diver = 1 - entropy_values
    weights = diver / np.sum(diver)
    return weights
    

entropy_weights(np.array([[10,200,0.5],
                          [20,300,0.7],
                          [30,250,0.9]]),optimal_value_indicator)

    
    
    

array([5.23520199e-03, 9.94763180e-01, 1.61762877e-06])

In [ ]:
RI = {1:0.00, 2:0.00, 3:0.58, 4:0.90, 5:1.12, 6:1.24, 7:1.32, 8:1.41, 9:1.45, 10:1.49}
def Normalize(A):
    """
    Normalize the pairwise comparison matrix A.
    """
    A = np.array(A, dtype=float)
    sj = np.sum(np.abs(A), axis=0)
    A = A / sj
    return A
def AHP(A):
    """
    Compute AHP weights and consistency ratio for square pairwise matrix A.
    Returns (weights, lambda_max, CI, CR).
    """
    A = np.array(A, dtype=float)
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix must be square (n x n).")
    #normalize
    A = Normalize(A)
    #print(f"Normalized A:\n{A}\n")
    # Eigen decomposition
    eigvals, eigvecs = np.linalg.eig(A)
    # pick eigenvector corresponding to largest eigenvalue (by real part)
    max_idx = np.argmax(eigvals.real)
    lambda_max = eigvals.real[max_idx]
    w = np.real(eigvecs[:, max_idx])
    w = w / w.sum()
    # Consistency
    CI = (lambda_max - n) / (n - 1) if n > 1 else 0.0
    RI_val = RI.get(n)
    CR = CI / RI_val if (RI_val is not None and RI_val != 0) else np.nan
    return w, lambda_max, CI, CR

In [ ]:
AHP([[1, 3, 5],
    [1/3, 1, 2],
    [1/5, 1/2, 1]])

(array([0.65199035, 0.2225573 , 0.12545235]),
 0.9999999999999998,
 -1.0,
 -1.7241379310344829)

In [ ]:
# indicator = [Energy consumption, emission,waste management,water consumption, temperature]
#energy_consumption_weights = [renewable energy ratio, energy efficiency, carbon emissions,renewable energy potential]
energy_consumption_weights = [[1., 36e6/73208, 343, 2744/365, 7.518]]
carbon_emission_of_trans = [[]]
waste_management = []
#water_usage = [Santa Clara, Atlanta  Georgia,  Inglewood CA ,Glendale,Las Vegas-NV,New Orleans, LA,Seatt,Nashville, Denever] 
water_usage = [(80+110)*3.78541178/2,149,None,280,830,(300+380)/2,190,(80+110)*3.78541178/2,125]  # gallons per person per day
climate = []



In [ ]:
def remove_unit(strin,unit):
    return strin.replace(unit,"")
    

**reading data from the dictionary**

In [ ]:
#import data
import os
from pathlib import Path
import pandas as pd
def read_csv_from(dir_or_url, filename):
    if isinstance(dir_or_url, Path):
        return pd.read_csv(dir_or_url / filename)
    else:
        # ensure no double slashes
        return pd.read_csv(f"{dir_or_url.rstrip('/')}/{filename}")
#local_data_dir = Path('/Users/daniel/Documents/Math_/Superbowl_vs_environment/data')


    

local_data_dir = Path('/Users/daniel/Documents/Math_/Superbowl_vs_environment/data')

# Raw base that points to the CSV files on GitHub (raw.githubusercontent)
raw_base = "https://raw.githubusercontent.com/maop0/Super-Bowl-Mathematical-Modelling/2139377798b53564aa659dc1185cfba3e5dd84c2/data"


data_dir = local_data_dir if local_data_dir.exists() else raw_base

energy_consumption_data = read_csv_from(data_dir, 'Energy_consmuption.csv')
waste_management_data = read_csv_from(data_dir, 'waste_management.csv')
climate_data = read_csv_from(data_dir, 'climate.csv')
emmision_data = read_csv_from(data_dir, 'transportation.csv')


**==========below we are processing energy_consumption_data==========**

In [ ]:
#process energy consumption data

for i in energy_consumption_data.columns:
    
    #removing [int]
    energy_consumption_data[i]= pd.DataFrame(map(lambda x:x.split("[")[0]if type(x) == str else x, energy_consumption_data[i]))
#energy_consumption_data
energy_consumption_data['可再生能源占比 (%)'] = pd.DataFrame(map(lambda x:remove_unit(x.split('[')[0],"%")if type(x) == str else x, energy_consumption_data['可再生能源占比 (%)']))
energy_consumption_data["可再生能源占比 (%)"][0] = energy_consumption_data["可再生能源占比 (%)"][0][1:] 
energy_consumption_data["可再生能源占比 (%)"][1] = energy_consumption_data["可再生能源占比 (%)"][1][1:] 
energy_consumption_data['可再生能源占比 (%)'] = pd.DataFrame(map(lambda x:float(x)/100, energy_consumption_data['可再生能源占比 (%)']))
for i in energy_consumption_data.columns[2:-2]:
    energy_consumption_data[i] = pd.DataFrame(map(float, energy_consumption_data[i]))
energy_consumption_data['平均风速 (km/h)'] = pd.DataFrame(map(lambda x:float(x.split('~')[1]) if x[0] == '~' else float(x), energy_consumption_data['平均风速 (km/h)']))

energy_consumption_data

,城市 (场馆),可再生能源占比 (%),场馆能效 (kWh/座位),电网碳强度 (kgCO₂/kWh),平均太阳辐照 (kWh/m²·日),平均风速 (km/h),说明（数据来源/估算）
0,Santa Clara（Levi’s Stadium）,1.000,0.530,0.184,5.91,8.1,场馆1150块太阳能板年发电可覆盖主场比赛用电
1,Atlanta（Mercedes-Benz Stadium）,1.000,0.507,0.305,5.29,9.4,场馆4000块太阳能板年发1.6GWh
2,Inglewood（SoFi Stadium）,0.570,0.513,0.184,6.21,11.0,场馆无自发电设施，采用加州电网平均可再生比例（约57%）
3,Glendale（State Farm Stadium）,0.140,0.568,0.288,6.59,10.9,采用亚利桑那州平均可再生比例（约14%）
4,Las Vegas（Allegiant Stadium）,1.000,0.554,0.286,6.51,12.7,场馆宣称使用“100%内华达州可再生电力”
5,New Orleans（Superdome）,0.030,0.492,0.420,5.38,15.2,采用路易斯安那州平均可再生比例（≈3%）
6,Seattle（Lumen Field）,0.757,0.524,0.113,4.12,6.6,采用华盛顿州平均可再生比例（约75.7%）
7,Denver（Empower Field）,0.430,0.473,0.358,5.93,13.2,采用科罗拉多州2024年可再生份额（43%）
8,Nashville（Nissan Stadium）,0.130,0.521,0.365,4.96,11.4,采用田纳西州平均可再生比例（≈13%）


**==========below we are processing emission data==========**

In [ ]:
#process emission data
emmision_data['公交站数量'] = pd.DataFrame(map(float, emmision_data['公交站数量']))
emmision_data['电车/地铁站数量'] = pd.DataFrame(map(float, emmision_data['电车/地铁站数量']))
emission_data = pd.DataFrame(emmision_data[emmision_data.columns[:-1]])

emission_data

,超级碗,年份,城市,场馆,公交站数量,电车/地铁站数量
0,Super Bowl L,2016.0,"Santa Clara, CA",Levi’s Stadium,5.0,3.0
1,Super Bowl LIII,2019.0,"Atlanta, GA",Mercedes-Benz Stadium,3.0,2.0
2,Super Bowl LVI,2022.0,"Inglewood, CA",SoFi Stadium,5.0,2.0
3,Super Bowl LVII,2023.0,"Glendale, AZ",State Farm Stadium,3.0,1.0
4,Super Bowl LVIII,2024.0,"Las Vegas, NV",Allegiant Stadium,2.0,0.0
5,Super Bowl LIX,2025.0,"New Orleans, LA",Caesars Superdome,6.0,1.0
6,Super Bowl LX,2026.0,"Santa Clara, CA",Levi’s Stadium,5.0,3.0
7,Super Bowl LXI,2027.0,"Inglewood, CA",SoFi Stadium,5.0,2.0
8,Super Bowl LXII,2028.0,"Atlanta, GA",Mercedes-Benz Stadium,3.0,2.0
9,Super Bowl LXIII,2029.0,To be determined,NaN,NaN,NaN


**==========below we are processing waste management data==========**

In [ ]:
#process waste management data
waste_management_data['总废弃物量'] = pd.DataFrame(map(lambda x: float(remove_unit(x, "t")) if type(x) == str else x, waste_management_data['总废弃物量']))
waste_management_data['总观众人数'] = pd.DataFrame(map(float, waste_management_data['总观众人数']))
waste_management_data['分类回收率'] = pd.DataFrame(map(lambda x: float(remove_unit(x, "%"))/100 if type(x) == str else x, waste_management_data['分类回收率']))


waste_management_data

,超级碗,年份,城市,总废弃物量,总观众人数,分类回收率
0,Super Bowl L,2016,"Santa Clara, CA",155.0,75000.0,0.75
1,Super Bowl LIII,2019,"Atlanta, GA",80.0,71000.0,0.90
2,Super Bowl LVI,2022,"Inglewood, CA",70.0,70000.0,0.90
3,Super Bowl LVII,2023,"Glendale, AZ",81.0,73000.0,0.92
4,Super Bowl LVIII,2024,"Las Vegas, NV",77.0,65000.0,0.90
5,Super Bowl LIX,2025,"New Orleans, LA",87.0,73200.0,0.91
6,Super Bowl LX,2026,"Santa Clara, CA",NaN,75000.0,NaN
7,Super Bowl LXI,2027,"Inglewood, CA",NaN,70000.0,NaN
8,Super Bowl LXII,2028,"Atlanta, GA",NaN,71000.0,NaN
9,Super Bowl LXIII,2029,To be determined,NaN,NaN,NaN


**==========below we are processing waste climate data==========**

In [ ]:
#process climate data
climate_data["平均每㎡入射太阳能"] = pd.DataFrame(map(lambda x: float(remove_unit(x, "kWh")) if type(x) == str else x, climate_data["平均每㎡入射太阳能"]))
climate_data["平均风速"] = pd.DataFrame(map(lambda x: float(remove_unit(x, " km/h")) if type(x) == str else x, climate_data["平均风速"]))
climate_data['平均温度'] = pd.DataFrame(map(lambda x: float(remove_unit(x, "℃") )if type(x) == str else x, climate_data["平均温度"]))

climate_data['城市']

0     Santa Clara, CA
1         Atlanta, GA
2       Inglewood, CA
3        Glendale, AZ
4       Las Vegas, NV
5     New Orleans, LA
6     Santa Clara, CA
7       Inglewood, CA
8         Atlanta, GA
9    To be determined
Name: 城市, dtype: object

Now we have been perepared, lets do the real work:


In [ ]:
#weights for the criteias of Energy consumption 
print(energy_consumption_data.columns[1:-1])
energy_consumption_matrix = np.array(energy_consumption_data[energy_consumption_data.columns[1:-1]])
energy_consumption_weights = entropy_weights(energy_consumption_matrix, benefit_indicator)
energy_consumption_weights

Index(['可再生能源占比 (%)', '场馆能效 (kWh/座位)', '电网碳强度 (kgCO₂/kWh)',
       '平均太阳辐照 (kWh/m²·日)', '平均风速 (km/h)'],
      dtype='object')


array([0.73266918, 0.00375127, 0.16703306, 0.02472947, 0.07181701])

In [ ]:
#weights for the criteias of Energy consumption 
emission_matrix = np.array(emission_data[emission_data.columns[-2:]])
# remove any row that has NaN in these two columns
emission_matrix = emission_matrix[~np.isnan(emission_matrix).any(axis=1)]
emission_weights = entropy_weights(emission_matrix,benefit_indicator)  # weights for the two criteria
emission_weights

array([0.21952617, 0.78047383])

In [ ]:
"""
             [[1. ,5. ,7.,4.   ,3],
              [1/5,1,  3,  1/2,1/3],
              [1/7,1/3,1,  1/5,1/4],
              [1/4,2. ,5,  1,  1/2],
              [1/3,3. ,4,  2,    1]] """
AHP_matric3 = np.array([     [1. ,5. ,7.,4.   ,3],
                            [1/5,1,  3,  1/3,1/5],
                            [1/7,1/3,1,  1/4,1/5],
                            [1/4,3. ,4,  1,  1/3],
                            [1/3,5. ,5,  3,    1]])
AHP_matric3 = AHP_matric3.T
AHP_matric3


array([[1.        , 0.2       , 0.14285714, 0.25      , 0.33333333],
       [5.        , 1.        , 0.33333333, 3.        , 5.        ],
       [7.        , 3.        , 1.        , 4.        , 5.        ],
       [4.        , 0.33333333, 0.25      , 1.        , 3.        ],
       [3.        , 0.2       , 0.2       , 0.33333333, 1.        ]])

In [ ]:
print(f'Normalized A:{Normalize(AHP_matric3)}')
print(array_latex_table(Normalize(AHP_matric3)))
AHP(AHP_matric3)[0]


Normalized A:[[0.05       0.04225352 0.07416564 0.02912621 0.02325581]
 [0.25       0.21126761 0.17305315 0.34951456 0.34883721]
 [0.35       0.63380282 0.51915946 0.46601942 0.34883721]
 [0.2        0.07042254 0.12978986 0.11650485 0.20930233]
 [0.15       0.04225352 0.10383189 0.03883495 0.06976744]]


array([0.05590931, 0.22234904, 0.5146012 , 0.12535282, 0.08178763])

In [ ]:
AHP_matric4 = np.array([[1, 1/7, 1/3, 1/5, 2],
                        [7, 1,   5,   3,   8],
                        [3, 1/5, 1,  1/3,  4],
                        [5, 1/3, 3,  1,    6],
                        [1/2,1/8,1/4,1/6,  1]])
AHP_matric4 = AHP_matric4.T
AHP_matric4

array([[1.        , 7.        , 3.        , 5.        , 0.5       ],
       [0.14285714, 1.        , 0.2       , 0.33333333, 0.125     ],
       [0.33333333, 5.        , 1.        , 3.        , 0.25      ],
       [0.2       , 3.        , 0.33333333, 1.        , 0.16666667],
       [2.        , 8.        , 4.        , 6.        , 1.        ]])

In [ ]:
print(f'Normalized A:{Normalize(AHP_matric4)}')
print(array_latex_table(Normalize(AHP_matric4)))
AHP(AHP_matric4)[0]

Normalized A:[[0.27202073 0.29166667 0.3515625  0.32608696 0.24489796]
 [0.0388601  0.04166667 0.0234375  0.02173913 0.06122449]
 [0.09067358 0.20833333 0.1171875  0.19565217 0.12244898]
 [0.05440415 0.125      0.0390625  0.06521739 0.08163265]
 [0.54404145 0.33333333 0.46875    0.39130435 0.48979592]]


array([0.27321162, 0.0468198 , 0.12226027, 0.06987235, 0.48783596])

**问题1：理解问题 - 环境影响指标体系**
**思路**
**建立三级指标体系：**


- 一级指标：能源、交通、废弃物、水资源、气候适应性


- 二级指标：细化的可测量指标 


- 三级指标：具体数据采集点 


**关键指标** 


1. **energy consumption**


    - 可再生能源比例（%）  


    - 场馆能源效率（kWh/座位）   


    - 电网清洁度（碳强度 gCO₂/kWh）   


    - 当地太阳能/风能潜力   


1. **carbon emission by transportation**
    - 平均观众出行距离（km）   


    - 公共交通覆盖率（%）   


    - 机场碳排放效率   


    - 电动车充电基础设施密度   


1. **waste management**
    - 回收率（%）   


    - 人均废弃物产生量（kg/人）   


    - 堆肥设施可用性   


    - 一次性塑料使用政策   


1. **water usage**
    - 水资源压力指数   


    - 雨水回收系统   


    - 节水技术应用率   


1. **climate**


    - 热岛效应强度   